In [1]:
import pyarrow.parquet as pq
import pyarrow as pa
import ujson

# 1. 处理Wiki数据

In [2]:
origin_wiki_file = './data/wiki.simple.txt'

liness = []
with open(origin_wiki_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

In [4]:
lines[0:5]

['数学：\n',
 '数学，是研究数量、结构以及空间等概念及其变化的一门学科，属于形式科学的一种。数学利用抽象化和逻辑推理，从计数、计算、量度、对物体形状及运动的观察发展而成。数学家们拓展这些概念，以公式化新的猜想，以及从选定的公理及定义出发，严谨地推导出一些定理。\n',
 '基础数学的知识与运用是生活中不可或缺的一环。对数学基本概念的完善，早在古埃及、美索不达米亚及古印度历史上的古代数学文本便可观见，而在古希腊那里有更为严谨的处理。从那时开始，数学的发展便持续不断地小幅进展，至16世纪的文艺复兴时期，因为新的科学发现和数学革新两者的交互，致使数学的加速发展，直至今日。数学并成为许多国家及地区的教育中的一部分。\n',
 '数学在许多领域都有应用，包括科学、工程、医学、经济学和金融学等。数学对这些领域的应用通常被称为应用数学，有时亦会激起新的数学发现，并导致全新学科的发展，例如物理学的实质性发展中建立的某些理论激发数学家对于某些问题的不同角度的思考。数学家也研究纯粹数学，就是数学本身的实质性内容，而不以任何实际应用为目标。许多研究虽然以纯粹数学开始，但其过程中也发现许多可用之处。\n',
 '词源。\n']

In [7]:
# 删除空行
new_lines = []
for line in lines:
    if line.strip() != '':
        new_lines.append(line)

In [8]:
print(len(lines), len(new_lines))

10103707 9804337


In [9]:
# with open('./data/wiki.simple_new.txt', 'w', encoding='utf-8') as f:
#     f.writelines(new_lines)

In [ ]:
tb = pa.Table.from_arrays([new_lines], names=['text'])
# compression='GZIP'
pq.write_table(table=tb, where='./data/wiki.simple.parquet', row_group_size=50000, data_page_size=50000, )

# 2. 处理训练数据

In [3]:
lines = []
with open('./data/Belle_open_source_1M.json', 'r', encoding='utf-8') as f:
    for line in f:
        item = ujson.loads(line)

        txt = ''
        if len(item['input'].strip()) != 0:
            txt = f"{item['instruction'].strip()}\n{item['input'].strip()}\n{item['output']}"
        else:
            txt = f"{item['instruction']}{item['output']}"
        
        if len(txt) == 0 or len(txt) >= 320: continue
        lines.append(
                txt
        )

In [4]:
print(len(lines), lines[0])

800037 判断给定的文章是否符合语法规则。如果不符合，请提供修改建议。
下面是一篇文章的开头: "为了探讨这个主题，本文将提供一系列数据和实例，以证明这一观点。"
这个开头符合语法规则。


In [6]:
tb = pa.Table.from_arrays([lines], names=['text'])
# compression='GZIP'
pq.write_table(table=tb, where='./data/instruction_data.parquet', row_group_size=20480, data_page_size=20480, )

# 3. 处理sft数据

In [6]:
lines = []
with open('./data/sft_0.8M_CN.json', 'r', encoding='utf-8') as f:
    for line in f:
        item = ujson.loads(line)

        txt = f"{item['instruction']}{item['output']}"
        
        if len(txt) == 0 or len(txt) >= 320: continue
        lines.append(
                item
        )

In [7]:
print(len(lines))

726475


In [8]:
tb = pa.Table.from_pylist(lines)
# compression='GZIP'
pq.write_table(table=tb, where='./data/sft_train_data.parquet', row_group_size=20480, data_page_size=20480, )